In [19]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
from webdriver_manager.chrome import ChromeDriverManager
import time

In [20]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\JeeMeng\.wdm\drivers\chromedriver\win32\95.0.4638.17\chromedriver.exe] found in cache


# NASA Mars News

In [3]:
url = 'https://redplanetscience.com/'
browser.visit(url)

In [4]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [5]:
result = soup.find('div', class_="list_text")

news_title = result.text
news_p = result.find('div',class_="article_teaser_body").text

print(f"news_title: {news_title}")
print(f"news_p: {news_p}")

news_title: 
October 26, 2021
My Culture, My Voice
In honor of Hispanic Heritage Month, Christina Hernandez, an instrument engineer on the Mars 2020 mission, talks about her childhood and journey to NASA.

news_p: In honor of Hispanic Heritage Month, Christina Hernandez, an instrument engineer on the Mars 2020 mission, talks about her childhood and journey to NASA.


# JPL Mars Space Images - Featured Image

In [6]:
fig_url = 'https://spaceimages-mars.com/'
browser.visit(fig_url)

In [7]:
fig_html = browser.html
mars_fig_soup = BeautifulSoup(fig_html, 'html.parser')



In [8]:
# the large image is within the figue element with class = lede
fig = mars_fig_soup.body.find("img", class_="headerimage fade-in")

# create the full url
featured_fig_url = fig_url+fig['src']

print(featured_fig_url)

https://spaceimages-mars.com/image/featured/mars3.jpg


# Mars Facts

In [9]:
facts_url ='https://galaxyfacts-mars.com/'
tables = pd.read_html(facts_url)
tables

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [10]:
table_df = tables[0]
table_df.columns = ["Description","Mars","Earth"]

table_df

,Description,Mars,Earth
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [11]:
mars_df = table_df.to_html("table.html", border="1",justify="left")

# Mars Hemispheres

In [21]:
hemis_url = 'https://marshemispheres.com/'
browser.visit(hemis_url)

In [22]:
hemis_html = browser.html
soup = BeautifulSoup(hemis_html, 'html.parser')

In [14]:
items = soup.find_all("div", class_="item")

main_url = "https://marshemispheres.com/"
hemisphere_url = []

for item in items:
    hemisphere_url.append(f"{main_url}{item.find('a', class_='itemLink')['href']}")

print(*hemisphere_url, sep = "\n")

https://marshemispheres.com/cerberus.html
https://marshemispheres.com/schiaparelli.html
https://marshemispheres.com/syrtis.html
https://marshemispheres.com/valles.html


In [23]:
hemis_fig_url = []
for url in hemisphere_url:
    browser.visit(url)
    hemis_html = browser.html 
    soup = BeautifulSoup(hemis_html, 'html.parser')
    fig_url = soup.find('img', class_="wide-image")['src']
    title = soup.find('h2', class_="title").text
   # Append the dictionary with the image url string and the hemisphere title to a list
    hemis_fig_url.append({"title":title,"fig_url":f"https://marshemispheres.com/{fig_url}"})
hemis_fig_url    

[{'title': 'Cerberus Hemisphere Enhanced',
  'fig_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'fig_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'fig_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'fig_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [26]:
mars_info = {
        "mars_news": {
            "news_title": news_title,
            "news_p": news_p,
            },
        "mars_img": featured_fig_url,
        "mars_fact": mars_df,
        "mars_hemisphere": himis_fig_url
    }
browser.quit()

print(mars_info)

{'mars_news': {'news_title': '\nOctober 26, 2021\nMy Culture, My Voice\nIn honor of Hispanic Heritage Month, Christina Hernandez, an instrument engineer on the Mars 2020 mission, talks about her childhood and journey to NASA.\n', 'news_p': 'In honor of Hispanic Heritage Month, Christina Hernandez, an instrument engineer on the Mars 2020 mission, talks about her childhood and journey to NASA.'}, 'mars_img': 'https://spaceimages-mars.com/image/featured/mars3.jpg', 'mars_fact': None, 'mars_hemisphere': [{'title': 'Cerberus Hemisphere Enhanced', 'fig_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'fig_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'fig_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'}, {'title': 